In [29]:
# data_loader outputs a dictionary of dictionary. 
# https://github.com/holos-aafc/Holos/blob/155851c8a5266cba65bae38361149aa8e47d280f/H.Core/Calculators/Carbon/ICBMSoilCarbonCalculator.cs#L270
# https://github.com/holos-aafc/Holos/blob/155851c8a5266cba65bae38361149aa8e47d280f/H.Core/Services/LandManagement/FieldResultsService.Initialization.cs#L204

In [38]:
class CropResidueCalculator:
    def __init__(self, data):        
        self.validate_input(data)
        self.data = data
        self.area = data["farm_data"]["area"]
        self.group = data["crop_group_params"]["group"].lower()
        self.crop_yield = data["farm_data"]["yield"]
        self.moisture = data["crop_parameters"]["moisture"]
        self.carbon_concentration = data["crop_group_params"]["carbon_concentration"]

        self.S_p = data["crop_group_params"]["S_p"]
        self.S_s = data["crop_group_params"]["S_s"]
        self.S_r = data["crop_group_params"]["S_r"]

        self.R_p = data["crop_parameters"]["R_p"]
        self.R_s = data["crop_parameters"]["R_s"]
        self.R_r = data["crop_parameters"]["R_r"]
        self.R_e = data["crop_parameters"]["R_e"]

        self.N_p = data["crop_parameters"]["N_p"]
        self.N_s = data["crop_parameters"]["N_s"]
        self.N_r = data["crop_parameters"]["N_r"]
        self.N_e = data["crop_parameters"]["N_e"]
    
    
    def validate_input(self, data):        
        if not isinstance(data["crop_group_params"]["group"], str):
            raise TypeError("group must be a string")
       
        if data["crop_group_params"]["group"].lower() not in ["annual", "perennial", "root", "cover", "silage"]:
            raise ValueError("group must be one of 'annual', 'perennial', 'root', 'cover', 'silage'")
    
        if data["farm_data"]["area"] < 0:
            raise ValueError("Area must be non-negative")
        
        if data["farm_data"]["yield"] < 0:
            raise ValueError("Yield must be non-negative")
        
        if not (0 <= data["crop_parameters"]["moisture"] <= 100):
            raise ValueError("Moisture must be between 0 and 100")

            
    def c_p(self):
        if abs(self.S_p - 100) < 1e-5:
            self.C_p = self.crop_yield * (1 - self.moisture / 100) * self.carbon_concentration
        else:
            self.C_p = (self.crop_yield + self.crop_yield * self.S_p / 100) * (1 - self.moisture / 100) * self.carbon_concentration
        
        return self.C_p

    def c_p_to_soil(self):
       
        self.C_p_to_soil = self.c_p() * (self.S_p / 100)
        
        return self.C_p_to_soil
   
    def c_s(self):

        if abs(self.R_p) < 1e-6:
            self.C_s = 0
        else:
            self.C_s = self.c_p() * (self.R_s / self.R_p) * (self.S_s / 100)
        
        return self.C_s

    def c_r(self):

        if abs(self.R_p) < 1e-6:
            self.C_r = 0
        else:
            self.C_r = self.c_p() * (self.R_r / self.R_p) * (self.S_r / 100)
        
        return self.C_r
    
    def c_e(self):
        
        if abs(self.R_p) < 1e-6:
            self.C_e = 0
        else:
            self.C_e = self.c_p() * (self.R_e / self.R_p)
        
        return self.C_e
    
    def grain_n(self):

        self.Grain_N = (self.c_p_to_soil() / 0.45) * (self.N_p / 1000)
        
        return self.Grain_N

    def straw_n(self):        
        self.Straw_N = (self.c_s() / 0.45) * (self.N_s / 1000)
        
        return self.Straw_N

    def root_n(self):
        self.Root_N = (self.c_r() / 0.45) * (self.N_r / 1000)
        
        return self.Root_N

    def exudate_n(self):
        
        self.Exudate_N = (self.c_e() / 0.45) * (self.N_e / 1000)
        
        return self.Exudate_N

    def above_ground_residue_n(self):

        if self.group in ["annual", "perennial"]:
            self.Above_Ground_Residue_N = self.grain_n() + self.straw_n()
        elif self.group == "root":
            self.Above_Ground_Residue_N = self.straw_n()
        elif self.group in ["cover", "silage"]:
            self.Above_Ground_Residue_N = self.grain_n()
        else:
            self.Above_Ground_Residue_N = 0
        
        return self.Above_Ground_Residue_N

    def below_ground_residue_n(self):
       
        if self.group == "annual":
            self.Below_Ground_Residue_N = self.root_n() + self.exudate_n()
        elif self.group == "perennial":
            self.Below_Ground_Residue_N = self.root_n() * (self.S_r / 100) + self.exudate_n()
        elif self.group == "root":
            self.Below_Ground_Residue_N = self.grain_n() + self.exudate_n()
        elif self.group in ["cover", "silage"]:
            self.Below_Ground_Residue_N = self.root_n() + self.exudate_n()
        else:
            self.Below_Ground_Residue_N = 0
        
        return self.Below_Ground_Residue_N

    def n_crop_residue(self):
        
        self.N_Crop_Residue = (self.above_ground_residue_n() + self.below_ground_residue_n()) * self.area
        
        return self.N_Crop_Residue


    def above_ground_carbon_input(self):

        if self.group == "root":
            self.Above_Ground_Carbon_Input = self.c_s()
        else:
            self.Above_Ground_Carbon_Input = self.c_p_to_soil() + self.c_s()
        
        return self.Above_Ground_Carbon_Input


        
    def below_ground_carbon_input(self):

        if self.group == "root":
            self.Below_Ground_Carbon_Input = self.c_p_to_soil() + self.c_e()
        else:
            self.Below_Ground_Carbon_Input = self.c_r() + self.c_e()
        
        return self.Below_Ground_Carbon_Input
    
    def get_crop_residue(self):
        
        all_data = {
            'C_p': self.c_p(),
            'above_ground_carbon_input': self.above_ground_carbon_input(),
            'below_ground_carbon_input': self.below_ground_carbon_input(),
            'above_ground_residue_n': self.above_ground_residue_n(),
            'below_ground_residue_n': self.below_ground_residue_n(),
            'n_crop_residue': self.n_crop_residue()
        }
        return all_data

In [39]:
data =  {
    "farm_data": 
            {
                "area": 10,
                "yield": 5000
            },
            
            "crop_group_params": 
            {
                "group": "root",
                "carbon_concentration": 0.45,
                "S_p": 0,
                "S_s": 100,
                "S_r": 100
            },
            
            "crop_parameters":
            {
                "moisture": 80,            
                "R_p": 0.626,
                "R_s": 0.357,
                "R_r": 0.01,
                "R_e": 0.007,
                "N_p": 10,
                "N_s": 29,
                "N_r": 10,
                "N_e": 10   
            },              
        }

**Holos result**: \
$S_p = 0$ \
$S_s = 100$ \
$S_r = 100$ \
$C_p = 450$ \
Above Ground Carbon Input (kg C ha^-1) = 256.629 \
Below Ground Carbon Input (kg C ha^-1) = 5.032

In [40]:
calculator = CropResidueCalculator(data)
calculator.get_crop_residue()

{'C_p': 449.9999999999999,
 'above_ground_carbon_input': 256.629392971246,
 'below_ground_carbon_input': 5.031948881789136,
 'above_ground_residue_n': 16.538338658146962,
 'below_ground_residue_n': 0.1118210862619808,
 'n_crop_residue': 166.5015974440894}

In [32]:
calculator = CropResidueCalculator(data)
print("Above Ground Carbon Input:", calculator.above_ground_carbon_input())
print("Below Ground Carbon Input:", calculator.below_ground_carbon_input())
print("C_p:", calculator.c_p())
print("C_p_to_soil:", calculator.c_p_to_soil())
print("C_s:", calculator.c_s())
print("C_r:", calculator.c_r())
print("C_e:", calculator.c_e())

Above Ground Carbon Input: 256.629392971246
Below Ground Carbon Input: 5.031948881789136
C_p: 449.9999999999999
C_p_to_soil: 0.0
C_s: 256.629392971246
C_r: 7.188498402555909
C_e: 5.031948881789136


In [33]:
data =  { # Potato Canada 
    "farm_data": 
            {
                "area": 10,
                "yield": 5000
            },
            
            "crop_group_params": 
            {
                "group": "root",
                "carbon_concentration": 0.45,
                "S_p": 0,
                "S_s": 100,
                "S_r": 100
            },
            
            "crop_parameters":
            {
                "moisture": 75,            
                "R_p": 0.736,
                "R_s": 0.239,
                "R_r": 0.015,
                "R_e": 0.01,
                "N_p": 15,
                "N_s": 20,
                "N_r": 10,
                "N_e": 10 
            },              
        }

**Holos result**: \
$S_p = 0$ \
$S_s = 100$ \
$S_r = 100$ \
$C_p = 563$ \
Above Ground Carbon Input (kg C ha^-1) = 182.66 \
Below Ground Carbon Input (kg C ha^-1) = 7.643

In [34]:
calculator = CropResidueCalculator(data)
print("Above Ground Carbon Input:", calculator.above_ground_carbon_input())
print("Below Ground Carbon Input:", calculator.below_ground_carbon_input())
print("C_p:", calculator.c_p())
print("C_p_to_soil:", calculator.c_p_to_soil())
print("C_s:", calculator.c_s())
print("C_r:", calculator.c_r())
print("C_e:", calculator.c_e())

Above Ground Carbon Input: 182.6596467391304
Below Ground Carbon Input: 7.642663043478262
C_p: 562.5
C_p_to_soil: 0.0
C_s: 182.6596467391304
C_r: 11.46399456521739
C_e: 7.642663043478262
